This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [126]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [127]:
# First, import the relevant modules
import requests

In [128]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=' + API_KEY)
# print(r.json())

In [129]:
print(r.json()['dataset_data']['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [130]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json()['dataset_data']['data'][:4])

[['2018-08-08', None, 68.75, 67.7, 68.65, None, 47181.0, 3225473.0, None, None, None], ['2018-08-07', None, 69.05, 67.85, 67.85, None, 55295.0, 3764837.0, None, None, None], ['2018-08-06', None, 69.8, 67.15, 68.9, None, 88362.0, 6085409.0, None, None, None], ['2018-08-03', None, 69.1, 67.3, 67.45, None, 110757.0, 7511670.0, None, None, None]]


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD)

define parameters for request, database and dataset codes and build query url

In [131]:
db_code = 'FSE'
ds_code = 'AFX_X'
start_dt = '2017-01-01'
end_dt = '2017-12-31'
query_url = 'https://www.quandl.com/api/v3/datasets/' 
query_url = query_url + db_code + '/' + ds_code + '/data.json?'
query_url = query_url + 'start_date=' + start_dt + '&end_date=' + end_dt
query_url = query_url + '&api_key=' + API_KEY

perform get request

In [132]:
r = requests.get(query_url)

#### 2. Convert the returned JSON object into a Python dictionary.

In [133]:
r_json = r.json()
data = r_json['dataset_data']['data']
keys = r_json['dataset_data']['column_names']

build a dictionary (r_dict indexed by a simple numeric index starting from 0 and incrementing by 1) of dictionaries (values in each nested dictionary indexed by the column names extracted in the variable keys above)

In [134]:
r_dict = { x : dict( zip(keys,value_lst) ) for x , value_lst in enumerate(data) }

In [157]:
# for example the fourth element of the dictionary indexed by key=3 is as follows
print( r_dict[3] )

{'Date': '2017-12-22', 'Open': 51.05, 'High': 51.5, 'Low': 50.92, 'Close': 51.32, 'Change': None, 'Traded Volume': 71165.0, 'Turnover': 3641949.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}


#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [136]:
# function to get the float value of any of the numeric fields in the dictionary
def get_float_value(d, key1, key2):
    try:
        return float(str(d[key1][key2]))
    except:
        return float("nan")

In [137]:
# fetch opening price using top level key
get_open_price = lambda key: get_float_value(r_dict, key, 'Open')

In [158]:
%%time
# get the top level key for the nested dictionary containing max. opening price
max_key = max( r_dict.keys(), key = get_open_price)

# get the top level key for the nested dictionary containing min. opening price
min_key = min( r_dict.keys(), key = get_open_price)

print('highest opening price: '+ str( get_open_price(max_key) ) )
print('lowest opening price: '+ str( get_open_price(min_key) ) )

highest opening price: 53.11
lowest opening price: 34.0
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 463 µs


#### 4. What was the largest change in any one day (based on High and Low price)?

In [169]:
# fetch High price using top level key
get_high_price = lambda key: get_float_value( r_dict, key, 'High' )

# fetch Low price using top level key
get_low_price = lambda key: get_float_value( r_dict, key, 'Low' )

# calculate High - Low to get daily change in price
get_daily_chg = lambda key: abs( get_high_price(key) - get_low_price(key) )

In [170]:
%%time
# get the top level key for the nested dictionary containing max. daily change
max_key = max( r_dict.keys(), key = get_daily_chg )

print('largest change in price in any one day: '+ str( get_daily_chg(max_key) ) )

largest change in price in any one day: 2.8100000000000023
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 470 µs


#### 5. What was the largest change between any two days (based on Closing Price)?

In [141]:
# fetch CLosing price using top level key
get_daily_close = lambda key: get_float_value( r_dict, key, 'Close')

# calculate change between consecutive days (consecutive nested dictionaries). use absolute value
get_nextday_chg = lambda key: abs( get_daily_close(key) - get_daily_close( int(key)+1 ) )

In [160]:
%%time
# get the top level key for the nested dictionary containing max. daily change
max_key = max( r_dict.keys(), key = get_nextday_chg )

print('largest change in price between any two days: '+ str( get_nextday_chg(max_key) ) )

largest change in price between any two days: 2.559999999999995
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 478 µs


#### 6. What was the average daily trading volume during this year?

In [147]:
# function to calculate average value 
def get_avg_value( d, val_by_key ):
    
    # function alias for get_float_value for d and key2 = val_by_key (key1 to be bound later)
    get_valuebykey = lambda key1: get_float_value( d = d, key1 = key1, key2 = val_by_key )    
    
    # calculate avg. by mapping the get_vbk_value lambda to every top level key 
    # to get the value for key=val_by_key for every record
    # and then calculating sum and dividing by total number of records
    return sum( map( get_valuebykey, d.keys() ) ) / len(d)

In [161]:
%%time
print('average daily volume: '+ str( get_avg_value( d = r_dict, val_by_key = 'Traded Volume') ) )

average daily volume: 89124.33725490196
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 265 µs


#### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [145]:
# function to calculate median value by finding the middle value after sorting by value
def get_median_value( d, sort_by_key ):
    length = len( d )
    
    # function alias for get_float_value for d and key2 = sort_by_key (key1 to be bound later)
    get_valuebykey = lambda key1: get_float_value( d = d, key1 = key1, key2 = sort_by_key )
    
    # sort top level keys by the value indexed by key key2
    sortd_kys = sorted( r_dict.keys(), key = get_valuebykey )
    
    if length == 0:
        return float("nan")
    elif length % 2 == 0:
        # mean of middle two values
        return ( get_valuebykey( sortd_kys[ int(length/2) - 1 ] ) + get_valuebykey( sortd_kys[ int(length/2) ] ) )/2
    else:
        # middle value
        return get_valuebykey( sortd_kys[ int( (length-1)/2 ) ] )

In [163]:
%%time
# print median for daily traded volumes
print('median daily volume: '+ str( get_median_value( d = r_dict, sort_by_key = 'Traded Volume' ) ) )

median daily volume: 76286.0
CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 292 µs
